In [1]:
import numpy as np
labelNames=np.array(['airplane','automobile','bird','cat','deer','dog','frog','horse', 'ship','truck'])
batch=128

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
import math
import time

import numpy as np
import tensorflow as tf

import cifar10_ACOL

FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_string('eval_dir', '/code/logs/cifar10_eval',
                           """Directory where to write event logs.""")
tf.app.flags.DEFINE_string('eval_data', 'test',
                           """Either 'test' or 'train_eval'.""")
tf.app.flags.DEFINE_string('checkpoint_dir', '/code/logs/cifar10_train',
                           """Directory where to read model checkpoints.""")
tf.app.flags.DEFINE_integer('eval_interval_secs', 60 * 5,
                            """How often to run the eval.""")
tf.app.flags.DEFINE_integer('num_examples', 10000,
                            """Number of examples to run.""")
tf.app.flags.DEFINE_boolean('run_once', True,
                         """Whether to run eval only once.""")

def eval_once(saver, summary_writer, top_k_op, summary_op, softmaxmat, images, labels):
  """Run Eval once.

  Args:
    saver: Saver.
    summary_writer: Summary writer.
    top_k_op: Top K op.
    summary_op: Summary op.
  """
  with tf.Session() as sess:
    ckpt = tf.train.get_checkpoint_state(FLAGS.checkpoint_dir)
    if ckpt and ckpt.model_checkpoint_path:
      print(ckpt.model_checkpoint_path)
      # Restores from checkpoint
      saver.restore(sess, ckpt.model_checkpoint_path)
      # Assuming model_checkpoint_path looks something like:
      #   /my-favorite-path/cifar10_train/model.ckpt-0,
      # extract global_step from it.
      global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
    else:
      print('No checkpoint file found')
      return

    # Start the queue runners.
    coord = tf.train.Coordinator()
    try:
      threads = []
      for qr in tf.get_collection(tf.GraphKeys.QUEUE_RUNNERS):
        threads.extend(qr.create_threads(sess, coord=coord, daemon=True,
                                         start=True))

      num_iter = int(math.ceil(FLAGS.num_examples / FLAGS.batch_size))
      true_count = 0  # Counts the number of correct predictions.
      total_sample_count = num_iter * FLAGS.batch_size
      step = 0
      #while step < num_iter and not coord.should_stop():
        #predictions = sess.run([top_k_op])
        #true_count += np.sum(predictions)
        #step += 1

      # Compute precision @ 1.
      #precision = true_count / total_sample_count
      #print('%s: precision @ 1 = %.3f' % (datetime.now(), precision))

      #summary = tf.Summary()
      #summary.ParseFromString(sess.run(summary_op))
      #summary.value.add(tag='Precision @ 1', simple_value=precision)
      #summary_writer.add_summary(summary, global_step)
    except Exception as e:  # pylint: disable=broad-except
      coord.request_stop(e)

    coord.request_stop()
    coord.join(threads, stop_grace_period_secs=10)
    
    return sess.run([softmaxmat, images, labels])


def evaluate():
  """Eval CIFAR-10 for a number of steps."""
  with tf.Graph().as_default() as g:
    # Get images and labels for CIFAR-10.
    eval_data = FLAGS.eval_data == 'test'
    images, labels,_ = cifar10_ACOL.inputs(eval_data=eval_data, raw=False,batch=batch)
    print(images.shape)
    # Build a Graph that computes the logits predictions from the
    # inference model.
    smStacked, stackedClusts, softmaxMat = cifar10_ACOL.inference(images,batch)
    print(smStacked.shape)
    # Calculate predictions.
    top_k_op = tf.nn.in_top_k(smStacked, labels, 1)

    # Restore the moving average version of the learned variables for eval.
    variable_averages = tf.train.ExponentialMovingAverage(
        cifar10_ACOL.MOVING_AVERAGE_DECAY)
    variables_to_restore = variable_averages.variables_to_restore()
    saver = tf.train.Saver(variables_to_restore)

    # Build the summary operation based on the TF collection of Summaries.
    summary_op = tf.summary.merge_all()

    summary_writer = tf.summary.FileWriter(FLAGS.eval_dir, g)

    while True:
      softmaxmatEval, imgEval, lblEval = eval_once(saver, summary_writer, top_k_op, summary_op,softmaxMat, images, labels)
      if FLAGS.run_once:
        break
      time.sleep(FLAGS.eval_interval_secs)
    
    return softmaxmatEval, imgEval, lblEval

In [3]:
softmaxmat, img, lbls = evaluate()

(128, 24, 24, 3)
(128, 2)
/code/logs/cifar10_train/model.ckpt-100000


OutOfRangeError: FIFOQueue '_2_batch/fifo_queue' is closed and has insufficient elements (requested 128, current size 0)
	 [[Node: batch = QueueDequeueManyV2[component_types=[DT_FLOAT, DT_INT32, DT_INT32], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/cpu:0"](batch/fifo_queue, batch/n)]]

Caused by op u'batch', defined at:
  File "/opt/conda/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/opt/conda/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/opt/conda/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/opt/conda/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/opt/conda/lib/python2.7/site-packages/zmq/eventloop/ioloop.py", line

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

In [ ]:
bgr2rgb = np.array([2,1,0])
#imgs_raw = img_raw[:,:,:,bgr2rgb]

In [ ]:
ind = 10
#plt.imshow(imgs_raw[ind],vmin=0,vmax=255,aspect='equal', interpolation='none') 
labelNames[lbls[ind]]

In [ ]:
cls = np.argmax(np.max(softmaxmat, axis=2),axis=1)

In [ ]:
cluster = []
for i in range(batch):
    cluster.append(softmaxmat[i,cls[i],:])
cluster = np.argmax(np.array(cluster),axis=1)

In [ ]:
classif = pd.DataFrame([labelNames[cls],cluster,range(batch)],['cls','cluster','index']).T
classifGrouped = classif.groupby(['cls','cluster']).count()
classifGrouped

In [ ]:
#plot 5 images of each cluster in class c:
classif
c='cat'

In [ ]:
t=[classif[(classif['cls']==c) & (classif['cluster']==i)]['index'] for i in range(5)]

In [ ]:
imgs_raw[t[clust]][index,:,:,:].shape

In [ ]:
f, plots = plt.subplots(5,4)
for clust in range(len(t)):
    for index in range(4):
        plots[clust,index].set_title(labelNames[lbls[t[clust].iloc[index]]])
        plots[clust,index].imshow(imgs_raw[t[clust]][index,:,:,:],vmin=0,vmax=255,aspect='equal', interpolation='none')